In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm
from random import shuffle
import torch
from torch import nn
import math
from glob import glob
import sys
import shutil  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


%matplotlib inline

In [ ]:
!pip install --upgrade --no-cache-dir gdown
import gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [ ]:
# Download training data
url = "https://drive.google.com/drive/folders/16ZaNLVBZgn7RYFLiF1jt-WcCF1aRb8aP"
gdown.download_folder(url, quiet=True, use_cookies=False, remaining_ok=True)

# Download test data
url = "https://drive.google.com/drive/folders/1dGwTm47UJSp0_qLl0n0VCUPMCu56xEZD"
gdown.download_folder(url, quiet=True, use_cookies=False, remaining_ok=True)

['/content/Test data/Aeolis Dorsa/images/ESP_072116_1740_RED.browse.jpg',
 '/content/Test data/Aeolis Dorsa/masks/ESP_072116_1740_RED.browse.png',
 '/content/Test data/Miyamoto Crater/images/ESP_016631_1770_RED.browse.jpg',
 '/content/Test data/Miyamoto Crater/images/ESP_074759_1855_RED.browse.jpg',
 '/content/Test data/Miyamoto Crater/masks/ESP_016631_1770_RED.browse.png',
 '/content/Test data/Miyamoto Crater/masks/ESP_074759_1855_RED.browse.png']

In [ ]:
url = "https://drive.google.com/drive/u/0/folders/1I1vxbZxmgR3OOPE7e-odfpcuCmJElICl"
gdown.download_folder(url, quiet=True, use_cookies=False, remaining_ok=True)

['/content/MIC Classifier/cnn_mic_classifier_20_0.31479.pth',
 '/content/MIC Classifier/cnn.pth',
 '/content/MIC Classifier/segmentation_model.pt']

In [ ]:
trainImages = sorted(glob('./Martian Inverted Channels' + '/*/images/*.jpg'))
if not os.path.exists('./TrainingData'):
  os.makedirs('./TrainingData')
if not os.path.exists('./TrainingData/TrainingPatches'):
  os.makedirs('./TrainingData/TrainingPatches')
if not os.path.exists('./TrainingData/TrainingPatches/images'):
  os.makedirs('./TrainingData/TrainingPatches/images')
if not os.path.exists('./TrainingData/TrainingPatches/masks'):
  os.makedirs('./TrainingData/TrainingPatches/masks')

In [ ]:
testImages = sorted(glob('./Test data' + '/*/images/*.jpg'))
if not os.path.exists('./TestingData'):
  os.makedirs('./TestingData')
if not os.path.exists('./TestingData/TestingPatches'):
  os.makedirs('./TestingData/TestingPatches')
if not os.path.exists('./TestingData/TestingPatches/images'):
  os.makedirs('./TestingData/TestingPatches/images')
if not os.path.exists('./TestingData/TestingPatches/masks'):
  os.makedirs('./TestingData/TestingPatches/masks')

In [ ]:
patchSize = 256

In [ ]:
def makePatches(dirName, images):

  patchHeight = patchSize
  patchWidth = patchSize
  patchList = []

  ImageSizes = []
  for ImagePath in images:
    image = cv2.imread(ImagePath)

    mask_path = ImagePath.replace('images', 'masks').replace('.jpg', '.png')
    mask = cv2.imread(mask_path)

    image_array = np.array(image)
    imageSize = image_array.shape
    imageHeight = imageSize[0]
    imageWidth = imageSize[1]

    ImageSizes.append([ImagePath[len(ImagePath)-30:-4],[imageWidth,imageHeight]])


    for y in range (0,imageHeight,patchSize):
      for x in range (0,imageWidth,patchSize):
        if ((y+patchHeight > imageHeight) and (x+patchWidth > imageWidth)):
          pat = image [y:imageHeight,x:imageWidth]
          patch = np.pad(pat,((0,y+patchHeight-imageHeight),(0,x+patchWidth-imageWidth),(0,0)),'reflect')
          maskPatch = mask [y:imageHeight,x:imageWidth]
          maskPatch = np.pad(maskPatch,((0,y+patchHeight-imageHeight),(0,x+patchWidth-imageWidth),(0,0)),'reflect')
        elif ((y+patchHeight <= imageHeight) and (x+patchWidth > imageWidth)):
          pat = image [y:y+patchHeight,x:imageWidth]
          patch = np.pad(pat,((0,0),(0,x+patchWidth-imageWidth),(0,0)),'reflect')
          maskPatch = mask [y:y+patchHeight,x:imageWidth]
          maskPatch = np.pad(maskPatch,((0,0),(0,x+patchWidth-imageWidth),(0,0)),'reflect')
        elif ((y+patchHeight > imageHeight) and (x+patchWidth <= imageWidth)):
          pat = image [y:imageHeight,x:x+patchWidth]
          patch = np.pad(pat,((0,y+patchHeight-imageHeight),(0,0),(0,0)),'reflect')
          maskPatch = mask [y:imageHeight,x:x+patchWidth]
          maskPatch = np.pad(maskPatch,((0,y+patchHeight-imageHeight),(0,0),(0,0)),'reflect')
        else:
          patch = image [y:y+patchHeight,x:x+patchWidth]
          maskPatch = mask [y:y+patchHeight,x:x+patchWidth]

        patchName = ImagePath[len(ImagePath)-30:-4]+"_"+str(int(y/patchSize))+"_"+str(int(x/patchSize))
        cv2.imwrite(dirName+'/images/'+patchName+'.jpg', patch)
        cv2.imwrite(dirName+'/masks/'+patchName+'.png', maskPatch)
        patchList.append(patch)
  return ImageSizes

In [ ]:
trainImages = sorted(glob('./Martian Inverted Channels' + '/*/images/*.jpg'))
testImages = sorted(glob('./Test data' + '/*/images/*.jpg'))
trainImageSizes = makePatches('./TrainingData/TrainingPatches', trainImages)
testImageSizes = makePatches('./TestingData/TestingPatches', testImages)

In [ ]:
print(len(trainImageSizes))
print(len(testImageSizes))

20
3


In [ ]:
print(testImageSizes)

[['ESP_072116_1740_RED.browse', [1703, 2412]], ['ESP_016631_1770_RED.browse', [1639, 2863]], ['ESP_074759_1855_RED.browse', [1519, 3926]]]


In [ ]:
import os
import numpy as np
import torch
import torch.utils.data
import torchvision.transforms as transforms
import PIL
import random
from scipy import ndimage


class segDataset(torch.utils.data.Dataset):
    def __init__(self, root, training, transform=None):
        super(segDataset, self).__init__()
        self.root = root
        self.training = training
        self.transform = transform
        self.IMG_NAMES = sorted(glob(self.root + '/images/*.jpg'))
        self.BGR_classes = {'Background' : [ 0, 0, 0],
                            'Inverted Channel' : [ 0, 0, 255]} # in BGR

        self.bin_classes = ['Background', 'Inverted Channel']


    def __getitem__(self, idx):
        
        img_path = self.IMG_NAMES[idx]
        mask_path = img_path.replace('images', 'masks').replace('.jpg', '.png')

        image = cv2.imread(img_path)
        mask = cv2.imread(mask_path)
        plt.imshow(mask)
        cls_mask = np.zeros(mask.shape)

        cls_mask[mask == self.BGR_classes['Background']] = self.bin_classes.index('Background')
        cls_mask[mask == self.BGR_classes['Inverted Channel']] = self.bin_classes.index('Inverted Channel')
        torch.set_printoptions(profile="full")
        #print("class mask",cls_mask)
        cls_mask = cls_mask[:,:,2] #removing nearby elements [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] -> [0,2,4,6,8]
        #print("---------------------------------------------------")
        #print(cls_mask)
        if self.training==True:
            if self.transform:
              image = transforms.functional.to_pil_image(image)
              image = self.transform(image)
              image = np.array(image)

        # image = cv2.resize(image, (128,128))/255.0
        # cls_mask = cv2.resize(cls_mask, (128,128))
        image = cv2.resize(image, (patchSize,patchSize))/255.0
        cls_mask = cv2.resize(cls_mask, (patchSize,patchSize)) 
        image = np.moveaxis(image, -1, 0)

        return torch.tensor(image).float(), torch.tensor(cls_mask, dtype=torch.int64), img_path


    def __len__(self):
        return len(self.IMG_NAMES)

In [ ]:
testDataset = segDataset('./TestingData', training = False)
print(testDataset.IMG_NAMES)

[]


In [ ]:
import os
import numpy as np
import torch
import torch.utils.data
import torchvision.transforms as transforms
import PIL
import random
from scipy import ndimage


class segTrainDataset(torch.utils.data.Dataset):
    def __init__(self, root, training, transform=None):
        super(segTrainDataset, self).__init__()
        self.root = root
        self.training = training
        self.transform = transform
        self.IMG_NAMES = sorted(glob(self.root + '/*/images/*.jpg'))
        self.BGR_classes = {'Background' : [ 0, 0, 0],
                            'Inverted Channel' : [ 0, 0, 255]} # in BGR

        self.bin_classes = ['Background', 'Inverted Channel']


    def __getitem__(self, idx):
        
        img_path = self.IMG_NAMES[idx]
        mask_path = img_path.replace('images', 'masks').replace('.jpg', '.png')

        image = cv2.imread(img_path)
        mask = cv2.imread(mask_path)
        plt.imshow(mask)
        cls_mask = np.zeros(mask.shape)

        cls_mask[mask == self.BGR_classes['Background']] = self.bin_classes.index('Background')
        cls_mask[mask == self.BGR_classes['Inverted Channel']] = self.bin_classes.index('Inverted Channel')
        torch.set_printoptions(profile="full")
        #print("class mask",cls_mask)
        cls_mask = cls_mask[:,:,2] #removing nearby elements [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] -> [0,2,4,6,8]
        #print("---------------------------------------------------")
        #print(cls_mask)
        if self.training==True:
            if self.transform:
              image = transforms.functional.to_pil_image(image)
              image = self.transform(image)
              image = np.array(image)
        # image = cv2.resize(image, (128,128))/255.0
        # cls_mask = cv2.resize(cls_mask, (128,128))
        image = cv2.resize(image, (patchSize,patchSize))/255.0
        cls_mask = cv2.resize(cls_mask, (patchSize,patchSize)) 
        image = np.moveaxis(image, -1, 0)

        return torch.tensor(image).float(), torch.tensor(cls_mask, dtype=torch.int64), img_path


    def __len__(self):
        return len(self.IMG_NAMES)

In [ ]:
dataset = segTrainDataset('./TrainingData', training = True)

len(dataset)

1986

In [ ]:
testDataset = segTrainDataset('./TestingData', training = True)

len(testDataset)

250

In [ ]:
train_dataset = dataset
test_dataset = testDataset

In [ ]:
BACH_SIZE = 2
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BACH_SIZE, shuffle=True, num_workers=2)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, shuffle=False, num_workers=2)

In [ ]:
count = 0
for batch_i, (x, y,z) in enumerate(train_dataloader):
  print(batch_i)
  print("----------------------------------------------------------")
  print(len(x))
  print("----------------------------------------------------------")
  print(len(y))
  print("----------------------------------------------------------")
  print(z)
  break
  count+=1
print(count)

0
----------------------------------------------------------
2
----------------------------------------------------------
2
----------------------------------------------------------
('./TrainingData/TrainingPatches/images/ESP_049346_1845_RED.browse_10_0.jpg', './TrainingData/TrainingPatches/images/ESP_048233_1770_RED.browse_11_1.jpg')
0


In [ ]:
BACH_SIZE = 16
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BACH_SIZE, shuffle=True, num_workers=2)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, shuffle=False, num_workers=2)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels , in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)


    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [ ]:
import torch.nn.functional as F
from torch.nn import ModuleList
import torch



class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, out_channels=32):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        bilinear = False

        self.inc = DoubleConv(n_channels, out_channels)
        self.down1 = Down(out_channels, out_channels * 2)
        self.down2 = Down(out_channels * 2, out_channels * 4)
        self.down3 = Down(out_channels * 4, out_channels * 8)
        factor = 2 if bilinear else 1
        self.down4 = Down(out_channels * 8, out_channels * 16 // factor)
        self.up1 = Up(out_channels * 16, out_channels * 8 // factor, bilinear)
        self.up2 = Up(out_channels * 8, out_channels * 4 // factor, bilinear)
        self.up3 = Up(out_channels * 4, out_channels * 2 // factor, bilinear)
        self.up4 = Up(out_channels * 2, out_channels, bilinear)
        self.outc = OutConv(out_channels, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return x1, x, logits


class MiniUNet(nn.Module):
    def __init__(self, n_channels, n_classes, out_channels=32):
        super(MiniUNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        bilinear = False

        self.inc = DoubleConv(n_channels, out_channels)
        self.down1 = Down(out_channels, out_channels*2)
        self.down2 = Down(out_channels*2, out_channels*4)
        self.down3 = Down(out_channels*4, out_channels*8)
        self.up1 = Up(out_channels*8, out_channels*4, bilinear)
        self.up2 = Up(out_channels*4, out_channels*2, bilinear)
        self.up3 = Up(out_channels*2, out_channels, bilinear)
        self.outc = OutConv(out_channels, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x = self.up1(x4, x3)
        x = self.up2(x, x2)
        x = self.up3(x, x1)
        logits = self.outc(x)
        return x1, x, logits


class Iternet(nn.Module):
    def __init__(self, n_channels, n_classes, out_channels=32, iterations=3):
        super(Iternet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.iterations = iterations

        # define the network UNet layer
        self.model_unet = UNet(n_channels=n_channels,
                               n_classes=n_classes, out_channels=out_channels)

        # define the network MiniUNet layers
        self.model_miniunet = ModuleList(MiniUNet(
            n_channels=out_channels*2, n_classes=n_classes, out_channels=out_channels) for i in range(iterations))

    def forward(self, x):
        x1, x2, logits = self.model_unet(x)
        for i in range(self.iterations):
            x = torch.cat([x1, x2], dim=1)
            _, x2, logits = self.model_miniunet[i](x)

        return logits


In [ ]:
# Updated
class WeightedDiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True, n_classes=2):
        super(WeightedDiceLoss, self).__init__()
        self.classes = n_classes

    def to_one_hot(self, tensor):
        n,h,w = tensor.size()
        one_hot = torch.zeros(n,self.classes,h,w).to(tensor.device).scatter_(1,tensor.view(n,1,h,w),1)
        return one_hot

    def forward(self, inputs, target):
       
        N = inputs.size()[0]
       

        # predicted probabilities for each pixel along channel
        inputs = F.softmax(inputs,dim=1)
        
        
        # Numerator Product
        target_oneHot = self.to_one_hot(target)
        weight_invertedChannels = 3
        weight_background = 1

        inter = 2*inputs * target_oneHot
        #print("inter size", inter.size())
        #(2.*intersection + smooth)/(inputs.sum() + target.sum() + smooth)
        intersection_channels = 0
        intersection_background = 0
        # smooth = 1

        for imageInd in range(N): 
          intersection_channels += inter[imageInd][1].view(-1).sum() 
          intersection_background += inter[imageInd][0].view(-1).sum() 

        # inter = intersection 
        #print("inter ", inter)
        #Denominator 
        union= inputs + target_oneHot
        ## Sum over all pixels N x C x H x W => N x C
        union_channels=0
        union_background = 0
        total2 = 0
        for imageInd in range(N):
          union_channels += union[imageInd][1].view(-1).sum() 
          union_background += union[imageInd][0].view(-1).sum() 

        # union = total2
        #print("union ", union)
        #union = union.view(N,self.classes,-1).sum(2)
        dice_channels =intersection_channels/union_channels
        dice_channels = dice_channels.mean()*0.8

        dice_background =intersection_background/union_background
        dice_background = dice_background.mean()*0.2

        weighted_dice = (dice_channels+ dice_background)

        loss = 1 - weighted_dice
        #print("loss ", loss)
        #print("loss mean ", loss.mean())
        ## Return average loss over classes and batch
        # return 1-loss.mean()
        return loss

In [ ]:
criterion = WeightedDiceLoss(n_classes=2).to(device)

In [ ]:
def acc(label, predicted):
  seg_acc = (y.cpu() == torch.argmax(pred_mask, axis=1).cpu()).sum() / torch.numel(y.cpu())
  return seg_acc

In [ ]:
min_loss = torch.tensor(float('inf'))
# model = UNet(n_channels=3, n_classes=2).to(device)
model = Iternet(n_channels=3, n_classes=2).to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)

In [ ]:
def getImageIndex(Array, imageID):
  imageIndex = -1
  for n in range (0,len(Array)):
    if Array[n][0] == imageID:
      imageIndex = n
      break
  
  return imageIndex

In [ ]:
def getImageSize(imageList, imageID):
  imageSize =[patchSize,patchSize]
  # imageSize =[128,128]
  for image in imageList:
    if image[0]==imageID:
      imageSize= image[1]
      break
  return imageSize

In [ ]:
# from pprint import pprint
# os.makedirs('./saved_models', exist_ok=True)

# N_EPOCHS = 1
# N_DATA = len(train_dataset)
# N_TEST = len(test_dataset)

# plot_losses = []
# scheduler_counter = 0
# prediction = []

# for epoch in range(N_EPOCHS):
#   # training
#   model.train()
#   loss_list = []
#   acc_list = []
#   predictions = []
#   for batch_i, (x, y, z) in enumerate(train_dataloader):

#       pred_mask = model(x.to(device))

#       loss = criterion(pred_mask, y.to(device))

#       optimizer.zero_grad()
#       loss.backward()
#       optimizer.step()
#       loss_list.append(loss.cpu().detach().numpy())
#       acc_list.append(acc(y,pred_mask).numpy())

#       sys.stdout.write(
#           "\r[Epoch %d/%d] [Batch %d/%d] [Loss: %f (%f)]"
#           % (
#               epoch,
#               N_EPOCHS,
#               batch_i,
#               len(train_dataloader),
#               loss.cpu().detach().numpy(),
#               np.mean(loss_list),
#           )
#       )
#   scheduler_counter += 1
#   # testing
#   model.eval()
#   val_loss_list = []
#   val_acc_list = []
#   for batch_i, (x, y, z) in enumerate(test_dataloader):
#       with torch.no_grad():    
#           pred_mask = model(x.to(device))
#       val_loss = criterion(pred_mask, y.to(device))
#       val_loss_list.append(val_loss.cpu().detach().numpy())
#       val_acc_list.append(acc(y,pred_mask).numpy())
    
#   print(' epoch {} - loss : {:.5f} - acc : {:.2f} - val loss : {:.5f} - val acc : {:.2f}'.format(epoch, 
#                                                                                                  np.mean(loss_list), 
#                                                                                                  np.mean(acc_list), 
#                                                                                                  np.mean(val_loss_list),
#                                                                                                  np.mean(val_acc_list)))
#   plot_losses.append([epoch, np.mean(loss_list), np.mean(val_loss_list)])
#   #  print("prediction ", prediction.size(), prediction.data[0])
#   compare_loss = np.mean(val_loss_list)
#   is_best = compare_loss < min_loss
#   if is_best == True:
#     scheduler_counter = 0
#     min_loss = min(compare_loss, min_loss)
#   torch.save(model.state_dict(), './saved_models/unet_epoch_{}_{:.5f}.pt'.format(epoch,np.mean(val_loss_list)))
  
#   if scheduler_counter > 5:
#     lr_scheduler.step()
#     print(f"lowering learning rate to {optimizer.param_groups[0]['lr']}")
#     scheduler_counter = 0


In [ ]:
from pprint import pprint
os.makedirs('./saved_models', exist_ok=True)

N_EPOCHS = 2
N_DATA = len(train_dataset)
N_TEST = len(test_dataset)

plot_losses = []
scheduler_counter = 0
prediction = []
# preds = []
for epoch in range(N_EPOCHS):
  # training
  model.train()
  loss_list = []
  acc_list = []
  # epochPreds = []
  for batch_i, (x, y, z) in enumerate(train_dataloader):
    # batchPreds = []
    for j in range(len(x)):

      pred_mask = model(x.to(device)[j:j+1])
      mask = torch.argmax(pred_mask, axis=1).cpu().detach().numpy()[0]
      # print("-----------------------------------")
      # print(z[j])
      # print("-----------------------------------")
      # imageIndex = getImageIndex(epochPreds,z[j][38:64])
      # imageSize = getImageSize(trainImageSizes,z[j][38:64])
      # cordinates = (z[j][65:-4]).split("_")
      # im = np.moveaxis(x.to(device)[j].cpu().detach().numpy(), 0, -1).copy()*255
      # im = im.astype(int)

      # gt_mask = y[j]
      # gt_mask = gt_mask.cpu().detach().numpy()
      # if imageIndex!=-1:
      #   (epochPreds[imageIndex][1]).append([cordinates,im,gt_mask,mask])
      # else:
      #   epochPreds.append([z[j][38:64],[[cordinates,im,gt_mask,mask]],imageSize])

      loss = criterion(pred_mask, y.to(device))

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_list.append(loss.cpu().detach().numpy())
      acc_list.append(acc(y,pred_mask).numpy())

      sys.stdout.write(
          "\r[Epoch %d/%d] [Batch %d/%d] [Loss: %f (%f)]"
          % (
              epoch,
              N_EPOCHS,
              batch_i,
              len(train_dataloader),
              loss.cpu().detach().numpy(),
              np.mean(loss_list),
          )
      )
    # epochPreds.append(batchPreds)
  scheduler_counter += 1
  # testing
  model.eval()
  val_loss_list = []
  val_acc_list = []
  for batch_i, (x, y, z) in enumerate(test_dataloader):
      with torch.no_grad():    
          pred_mask = model(x.to(device))
      val_loss = criterion(pred_mask, y.to(device))
      val_loss_list.append(val_loss.cpu().detach().numpy())
      val_acc_list.append(acc(y,pred_mask).numpy())
    
  print(' epoch {} - loss : {:.5f} - acc : {:.2f} - val loss : {:.5f} - val acc : {:.2f}'.format(epoch, 
                                                                                                 np.mean(loss_list), 
                                                                                                 np.mean(acc_list), 
                                                                                                 np.mean(val_loss_list),
                                                                                                 np.mean(val_acc_list)))
  plot_losses.append([epoch, np.mean(loss_list), np.mean(val_loss_list)])
  #  print("prediction ", prediction.size(), prediction.data[0])
  compare_loss = np.mean(val_loss_list)
  is_best = compare_loss < min_loss
  if is_best == True:
    # preds = epochPreds
    scheduler_counter = 0
    min_loss = min(compare_loss, min_loss)
  torch.save(model.state_dict(), './saved_models/unet_epoch_{}_{:.5f}.pt'.format(epoch,np.mean(val_loss_list)))
  
  if scheduler_counter > 5:
    lr_scheduler.step()
    print(f"lowering learning rate to {optimizer.param_groups[0]['lr']}")
    scheduler_counter = 0


[Epoch 0/2] [Batch 124/125] [Loss: 0.800507 (0.792080)] epoch 0 - loss : 0.79208 - acc : 0.94 - val loss : 0.80291 - val acc : 0.96
[Epoch 1/2] [Batch 124/125] [Loss: 0.800000 (0.802734)] epoch 1 - loss : 0.80273 - acc : 0.97 - val loss : 0.80499 - val acc : 0.96


In [ ]:
# os.makedirs('./saved_models', exist_ok=True)

# N_EPOCHS = 20
# N_DATA = len(train_dataset)
# N_TEST = len(test_dataset)

# plot_losses = []
# scheduler_counter = 0
# prediction = []
# preds = []
# for epoch in range(N_EPOCHS):
#   # training
#   model.train()
#   loss_list = []
#   acc_list = []
#   epochPreds = []
#   for batch_i, (x, y, z) in enumerate(train_dataloader):
#     # batchPreds = []
#     for j in range(len(x)):

#       pred_mask = model(x.to(device)[j:j+1])
#       mask = torch.argmax(pred_mask, axis=1).cpu().detach().numpy()[0]
#       # print("-----------------------------------")
#       # print(z[j])
#       # print("-----------------------------------")
#       imageIndex = getImageIndex(epochPreds,z[j][38:64])
#       imageSize = getImageSize(trainImageSizes,z[j][38:64])
#       cordinates = (z[j][65:-4]).split("_")
#       im = np.moveaxis(x.to(device)[j].cpu().detach().numpy(), 0, -1).copy()*255
#       im = im.astype(int)

#       gt_mask = y[j]
#       gt_mask = gt_mask.cpu().detach().numpy()
#       if imageIndex!=-1:
#         (epochPreds[imageIndex][1]).append([cordinates,im,gt_mask,mask])
#       else:
#         epochPreds.append([z[j][38:64],[[cordinates,im,gt_mask,mask]],imageSize])

#       loss = criterion(pred_mask, y.to(device))

#       optimizer.zero_grad()
#       optimizer.step()
#       loss_list.append(loss.cpu().detach().numpy())

#       sys.stdout.write(
#           "\r[Epoch %d/%d] [Batch %d/%d] [Loss: %f (%f)]"
#           % (
#               epoch,
#               N_EPOCHS,
#               batch_i,
#               len(train_dataloader),
#               loss.cpu().detach().numpy(),
#               np.mean(loss_list),
#           )
#       )
#     # epochPreds.append(batchPreds)
#   scheduler_counter += 1
#   # testing
#   model.eval()
#   val_loss_list = []
#   val_acc_list = []
#   for batch_i, (x, y, z) in enumerate(test_dataloader):
#       with torch.no_grad():    
#           pred_mask = model(x.to(device))
#       val_loss = criterion(pred_mask, y.to(device))
#       val_loss_list.append(val_loss.cpu().detach().numpy())
#       val_acc_list.append(acc(y,pred_mask).numpy())
    
#   print(' epoch {} - loss : {:.5f} - acc : {:.2f} - val loss : {:.5f} - val acc : {:.2f}'.format(epoch, 
#                                                                                                  np.mean(loss_list), 
#                                                                                                  np.mean(acc_list), 
#                                                                                                  np.mean(val_acc_list)))
#   plot_losses.append([epoch, np.mean(loss_list), np.mean(val_loss_list)])
#   #  print("prediction ", prediction.size(), prediction.data[0])
#   compare_loss = np.mean(val_loss_list)
#   is_best = compare_loss < min_loss
#   if is_best == True:
#     preds = epochPreds
#     scheduler_counter = 0
#     min_loss = min(compare_loss, min_loss)
#   torch.save(model.state_dict(), './saved_models/unet_epoch_{}_{:.5f}.pt'.format(epoch,np.mean(val_loss_list)))
  
#   if scheduler_counter > 5:
#     lr_scheduler.step()
#     print(f"lowering learning rate to {optimizer.param_groups[0]['lr']}")
#     scheduler_counter = 0


In [ ]:
# model.load_state_dict(torch.load('/content/saved_models/unet_epoch_0_0.80383.pt'))

In [ ]:
# import classifier model
# url = "https://drive.google.com/drive/u/0/folders/1I1vxbZxmgR3OOPE7e-odfpcuCmJElICl"
# gdown.download_folder(url, quiet=True, use_cookies=False, remaining_ok=True)

# model.load_state_dict(torch.load('/content/saved_models/unet_epoch_18_0.89815.pt'))
model.load_state_dict(torch.load('/content/MIC Classifier/segmentation_model.pt'))
model.eval()

In [ ]:
predictions = []
# model.eval()
m = nn.Softmax(dim=1)
# print(testImageSizes)
for batch_i, (x, y,z) in enumerate(train_dataloader):
    for j in range(len(x)):
      result = model(x.to(device)[j:j+1])
      # mask = torch.argmax(result, axis=1).cpu().detach().numpy()[0]
      mask1 = m(torch.from_numpy(result.cpu().detach().numpy()[0]))
      mask = torch.argmax(mask1, axis=1).cpu().detach().numpy()
      # mask = result.cpu().detach().numpy()[0]
      imageIndex = getImageIndex(predictions,z[j][38:64])
      # print("list ",z[j])
      imageSize = getImageSize(trainImageSizes,z[j][38:64])
      cordinates = (z[j][65:-4]).split("_")
      
      im = np.moveaxis(x.to(device)[j].cpu().detach().numpy(), 0, -1).copy()*255
      im = im.astype(int)
      
      gt_mask = y[j]
      gt_mask = gt_mask.cpu().detach().numpy()
      if imageIndex!=-1:
        (predictions[imageIndex][1]).append([cordinates,im,gt_mask,mask])
      else:
        predictions.append([z[j][38:64],[[cordinates,im,gt_mask,mask]],imageSize])

        # imageIndex = getImageIndex(epochPreds,z[j][38:64])
#       imageSize = getImageSize(trainImageSizes,z[j][38:64])
#       cordinates = (z[j][65:-4]).split("_")
#       im = np.moveaxis(x.to(device)[j].cpu().detach().numpy(), 0, -1).copy()*255
#       im = im.astype(int)

#       gt_mask = y[j]
#       gt_mask = gt_mask.cpu().detach().numpy()
#       if imageIndex!=-1:
#         (epochPreds[imageIndex][1]).append([cordinates,im,gt_mask,mask])
#       else:
#         epochPreds.append([z[j][38:64],[[cordinates,im,gt_mask,mask]],imageSize])

In [ ]:
a = torch.randn(4, 4)
a

tensor([[ 0.4036, -1.5211,  0.3832, -0.4665],
        [ 0.1664,  2.1122, -0.7550, -0.2734],
        [ 1.2436,  0.8844,  1.9544,  0.5287],
        [-0.7093, -0.7989,  1.2137, -0.1271]])

In [ ]:
m = nn.Softmax(dim=1)
output = m(a)
output

tensor([[0.3930, 0.0573, 0.3850, 0.1646],
        [0.1106, 0.7741, 0.0440, 0.0712],
        [0.2368, 0.1653, 0.4820, 0.1158],
        [0.0948, 0.0867, 0.6488, 0.1697]])

In [ ]:
for e in predictions:
  print(e[1][0][3])
  break

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
l = "./TrainingData/TrainingPatches/images/PSP_007704_1765_RED.browse_3_4.jpg"
l[65:-4]

'3_4'

In [ ]:
from operator import itemgetter

def constructImageRow(rowPatches):
  sortedPatches = sorted(rowPatches, key=itemgetter(0))
  rowImge =  torch.from_numpy(sortedPatches[0][1])
  for pt in sortedPatches[1:]:
    rowImge = torch.cat(((rowImge),torch.from_numpy(pt[1])), 1)
  return rowImge

In [ ]:
# from operator import itemgetter

# def reconstructImage(ImagePatches,type):
#   variable =True
#   current_row = 0
#   rows =[]
#   imageSize = ImagePatches[2]
#   # print("size ",imageSize  )
#   imageWidth = imageSize[0]
#   imageHeight = imageSize[1]
#   predictionPatches = sorted(ImagePatches[1], key=int(itemgetter(0)))
#   for i in predictionPatches:
#     print(i[0])
#   for n in range(0,len(predictionPatches)):
#     y_cor = int(predictionPatches[n][0][0])
#     x_cor = int(predictionPatches[n][0][1])
    
#     if (current_row==y_cor and x_cor==0):
#       rowPatches = [[x_cor,predictionPatches[n][type]]]
#       # rowImage = torch.from_numpy(predictionPatches[n][type])
#     elif (current_row==y_cor and x_cor!=0):
#       rowPatches.append([x_cor,predictionPatches[n][type]])
#       # rowImage = torch.cat(((rowImage),torch.from_numpy(predictionPatches[n][type])), 1)
#     else:
#       # numPatchesInRow = int(predictionPatches[n-1][0][1])+1
#       numPatchesInRow = len(rowPatches)+1
#       newRow = constructImageRow(rowPatches)
#       rows.append([current_row,newRow])
#       rowPatches = [[x_cor,predictionPatches[n][type]]]
#       # rowImage = torch.from_numpy(predictionPatches[n][type])
#       current_row = y_cor
#     if(n+1==len(predictionPatches)):
#       newRow = constructImageRow(rowPatches)
#       rows.append([y_cor,newRow])
#       variable = False

  
#   rows = sorted(rows, key=itemgetter(0))
  
#   fullImage = rows[0][1]
#   # print("y_cor", rows[0][0])
#   # plt.imshow(rows[0][1])
#   # plt.show()
#   for row in rows[1:]:
#     fullImage = torch.cat(((fullImage),(row[1])), 0)
#   croppedImage = fullImage[0:imageHeight,0:imageWidth]

#   return croppedImage

# # ImageOut = reconstructImage(predictions[2])
# # plt.figure(figsize=(15,15))   
# # plt.imshow(ImageOut)

In [ ]:
from operator import itemgetter

def reconstructImage(ImagePatches,type):
  variable =True
  current_row = 0
  rows =[]
  imageSize = ImagePatches[2]
  # print("size ",imageSize  )
  imageWidth = imageSize[0]
  imageHeight = imageSize[1]
  # predictionPatches = ImagePatches[1]
  predictionPatches = sorted(ImagePatches[1], key=itemgetter(0))
  for n in range(0,len(predictionPatches)):
    y_cor = int(predictionPatches[n][0][0])
    x_cor = int(predictionPatches[n][0][1])
    
    if (current_row==y_cor and x_cor==0):
      rowPatches = [[x_cor,predictionPatches[n][type]]]
      # rowImage = torch.from_numpy(predictionPatches[n][type])
    elif (current_row==y_cor and x_cor!=0):
      rowPatches.append([x_cor,predictionPatches[n][type]])
      # rowImage = torch.cat(((rowImage),torch.from_numpy(predictionPatches[n][type])), 1)
    else:
      # numPatchesInRow = int(predictionPatches[n-1][0][1])+1
      numPatchesInRow = len(rowPatches)+1
      newRow = constructImageRow(rowPatches)
      rows.append([current_row,newRow])
      rowPatches = [[x_cor,predictionPatches[n][type]]]
      # rowImage = torch.from_numpy(predictionPatches[n][type])
      current_row = y_cor
    if(n+1==len(predictionPatches)):
      newRow = constructImageRow(rowPatches)
      rows.append([y_cor,newRow])
      variable = False

  
  rows = sorted(rows, key=itemgetter(0))
  
  fullImage = rows[0][1]
  # print("y_cor", rows[0][0])
  # plt.imshow(rows[0][1])
  # plt.show()
  for row in rows[1:]:
    fullImage = torch.cat(((fullImage),(row[1])), 0)
  croppedImage = fullImage[0:imageHeight,0:imageWidth]

  return croppedImage

# ImageOut = reconstructImage(predictions[2])
# plt.figure(figsize=(15,15))   
# plt.imshow(ImageOut)

In [ ]:
# print(preds[0][0][0][1][0][1])
print(len(preds[0]))

20


In [ ]:
for element in preds[0]:
  print(len(element[1]))
  break

96


In [ ]:
for element in preds[0]:
  print(element[2])
  break

[1519, 3947]


In [ ]:
from skimage import io, morphology
from sklearn.metrics import f1_score   
from sklearn.metrics import jaccard_score

# for element in preds[0]:
for element in predictions:
  image = reconstructImage(element,1)
  annotatedMask = reconstructImage(element,2)
  predictionMask = reconstructImage(element,3)
  # f1_scored = f1_score(annotatedMask,predictionMask, average='weighted',zero_division=1)
  # jaccard = jaccard_score(annotatedMask,predictionMask,average='weighted',zero_division=1)
  # print("F1 Score = ",f1_scored)
  # print("Jaccaard Score = ",jaccard)
  plt.figure(figsize=(14,14))
  plt.subplot(1,3,1)
  plt.imshow(image)
  annotated= annotatedMask.cpu().detach().numpy()
  annotated[annotated<0.5] = 255
  annotated[annotated==1] = 0
  plt.subplot(1,3,2)
  plt.imshow(annotatedMask ,cmap = 'gray')
  
  
  numpyPred = predictionMask.cpu().detach().numpy()
  numpyPred[numpyPred<0.5] = 255
  numpyPred[numpyPred==1] = 0
  plt.subplot(1,3,3)
  plt.imshow(numpyPred, cmap = 'gray')
  
  plt.show()
  

TypeError: ignored

In [ ]:
# plot loss
plot_losses = np.array(plot_losses)
plt.plot(plot_losses[:,0], plot_losses[:,1], color='b', linewidth=4)
plt.plot(plot_losses[:,0], plot_losses[:,2], color='r', linewidth=4)
plt.title('Dice Loss', fontsize=20)
plt.xlabel('epoch',fontsize=20)
plt.ylabel('loss',fontsize=20)
plt.grid()
plt.legend(['training', 'validation']) # using a named size
plt.show()

In [ ]:
model.load_state_dict(torch.load('/content/saved_models/unet_epoch_24_0.78482.pt'))

In [ ]:
predictions = []
model.eval()
# print(testImageSizes)
for batch_i, (x, y,z) in enumerate(test_dataloader):
    for j in range(len(x)):
      result = model(x.to(device)[j:j+1])
      mask = torch.argmax(result, axis=1).cpu().detach().numpy()[0]
      imageIndex = getImageIndex(predictions,z[j][36:62])
      # print("list ",z[j])
      imageSize = getImageSize(testImageSizes,z[j][36:62])
      cordinates = (z[j][63:-4]).split("_")
      
      im = np.moveaxis(x.to(device)[j].cpu().detach().numpy(), 0, -1).copy()*255
      im = im.astype(int)
      
      gt_mask = y[j]
      gt_mask = gt_mask.cpu().detach().numpy()
      if imageIndex!=-1:
        (predictions[imageIndex][1]).append([cordinates,im,gt_mask,mask])
      else:
        predictions.append([z[j][36:-8],[[cordinates,im,gt_mask,mask]],imageSize])